<h1 align="center" style="background-color:#616161;color:white">RNN Model</h1>

Adapted from: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py

<h3 style="background-color:#616161;color:white">0. Setup</h3>

<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Input Parameters</div>

In [14]:
# Root path
#root = "C:/DS/Github/MusicRecommendation"  # BA, Windows
root = "/home/badrul/git/EventPrediction" # BA, Linux

<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Common Libraries</div>

In [15]:
# Core
import numpy as np
import pandas as pd
from IPython.core.debugger import Tracer    # Used for debugging
import logging

# File and database management
import csv
import os
import sys
import json
import sqlite3
from pathlib import Path

# Date/Time
import datetime
import time
#from datetime import timedelta # Deprecated

# Visualization
import matplotlib.pyplot as plt             # Quick
%matplotlib inline

# Misc
import random

#-------------- Custom Libs -----------------#
os.chdir(root)

# Import the codebase module
fPath = root + "/1_codemodule"
if fPath not in sys.path: sys.path.append(fPath)

# Custom Libs
import coreCode as cc
import lastfmCode as fm

<div style="background-color:white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Page Specific Libraries</div>

In [16]:
# Data science (comment out if not needed)
#from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.python.framework import ops
ops.reset_default_graph()
from sklearn import metrics
from sklearn import preprocessing

<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Declare Functions</div>

In [147]:
def getTrainAndTestData():
    con = sqlite3.connect(dbPath)
    c = con.cursor()

    # Get list of UserIDs 
    users = pd.read_sql_query("Select UserID from tblUsers Where tblUsers.TestUser = 0",con)

    fieldList="t, UserID, HrsFrom6pm, isSun,isMon,isTue,isWed,isThu,isFri,isSat,t1,t2,t3,t4,t5,t10,t12hrs,t24hrs,t1wk,t2wks,t3wks,t4wks"
    trainDf=pd.DataFrame(columns=[fieldList])  # Create an emmpty df
    testDf=pd.DataFrame(columns=[fieldList])  # Create an emmpty df
    periodsInAMonth=int(60/periodGranularity)*24*7*4

    totalRows=0
    
    for user in users.itertuples():
        # Get training dataset
        SqlStr="SELECT {} from tblTimeSeriesData where UserID = {}".format(fieldList,user.userID)
        df = pd.read_sql_query(SqlStr, con)
        totalRows += len(df)
    
        # Cut-off 1
        k = random.randint(periodsInAMonth, len(df))
        
        testDf = testDf.append(df.iloc[k:k+periodsInAMonth])[df.columns.tolist()]

        tmp = df.drop(df.index[k:k+periodsInAMonth])

        # Cut-off 2
        k = random.randint(periodsInAMonth, len(tmp))
        testDf = testDf.append(tmp.iloc[k:k+periodsInAMonth])[df.columns.tolist()]
        trainDf = trainDf.append(tmp.drop(tmp.index[k:k+periodsInAMonth]))[df.columns.tolist()]

    if len(trainDf)+len(testDf) == totalRows:
        print('Ok')
    else:
        print("Incorrect. Total Rows = {}. TestDf+TrainDf rows = {}+{}={}".format(totalRows,len(testDf),len(trainDf),len(testDf)+len(trainDf)))
        
    return trainDf, testDf

def getHiddenTestUsers(firstNPerc=1.0):
    con = sqlite3.connect(dbPath)
    c = con.cursor()

    # Get list of UserIDs 
    users = pd.read_sql_query("Select UserID from tblUsers Where tblUsers.TestUser = 1",con)

    fieldList="t, PeriodID, UserID, HrsFrom6pm, isSun,isMon,isTue,isWed,isThu,isFri,isSat,t1,t2,t3,t4,t5,t10,t12hrs,t24hrs,t1wk,t2wks,t3wks,t4wks"
    testDf=pd.DataFrame(columns=[fieldList])  # Create an emmpty df
    periodsInAMonth=int(60/periodGranularity)*24*7*4

    totalRows=0
    
    for user in users.itertuples():
        # Get training dataset
        SqlStr="SELECT {} from tblTimeSeriesData where UserID = {}".format(fieldList,user.userID)
    
        df = pd.read_sql_query(SqlStr, con)
        df["PeriodID"] = df["PeriodID"].astype(int)
        df.sort_values(['PeriodID'])
        totalRows += len(df)
        # Caluclate period cutt-off
        cutoff = int(len(df)*firstNPerc)
        testDf = testDf.append(df.iloc[0:cutoff])[df.columns.tolist()]
 
    testDf["PeriodID"] =  testDf["PeriodID"].astype(int)
    testDf["UserID"] =  testDf["UserID"].astype(int)
    testDf.sort_values(['UserID','PeriodID'], inplace=True)
    return testDf


SyntaxError: invalid syntax (<ipython-input-147-a18284c12140>, line 23)

<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Load settings</div>

In [87]:
settingsDict =  cc.loadSettings()
dbPath = root + settingsDict['mainDbPath']
fmSimilarDbPath = root + settingsDict['fmSimilarDbPath']
fmTagsDbPath = root + settingsDict['fmTagsDbPath']
trackMetaDbPath = root + settingsDict['trackmetadata']
periodGranularity = int(settingsDict['periodGranularity'])

<h3 style="background-color:#616161;color:white">1. Load data</h3>

In [150]:
# Train Data
trainDf,testDf = getTrainAndTestData()
xTrain = trainDf.drop(['t','UserID'], 1).values
yTrain_onehot = pd.get_dummies(trainDf['t']).values.astype(float) # One-Hot version

# Test data
xTest= testDf.drop(['t','UserID'], 1).values
yTest = testDf['t'].values.astype(int)
#yTest = np.array([1 if y==1 else -1 for y in y_vals_test])
yTest=yTest.reshape(len(yTest),1)

# One-Hot version
yTest_onehot = pd.get_dummies(testDf['t']).values.astype(float)

# Get hidden users data
testDf = getHiddenTestUsers(firstNPerc=0.5)  # Get the first half of everyones history
xTest2= testDf.drop(['t','UserID','PeriodID'], 1).values
yTest2 = testDf['t'].values.astype(int)
yTest2=yTest2.reshape(len(yTest2),1)

# One-Hot version
yTest2_onehot = pd.get_dummies(testDf['t']).values.astype(float)

Ok


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Confirm dimensions</div>

In [89]:
np.shape(xTrain),np.shape(yTrain_onehot)

((1354792, 20), (1354792, 2))

In [90]:
np.shape(xTest), np.shape( yTest),np.shape(yTest_onehot)

((82995, 20), (82995, 1), (82995, 2))

<h3 style="background-color:#616161;color:white">3. RNN Model</h3>

<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Define the model</div>

In [51]:
tf.reset_default_graph()

# Parameters
learning_rate = 0.001

display_step = 20
training_iters = np.shape(x_vals)[0]-batch_size
# Network Parameters
n_input = np.shape(x_vals)[1]
n_steps = 1 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 2

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Train the model</div>

In [52]:
training_iteration = 20
# Launch the graph
sess = tf.Session()

# Initializing the variables
init = tf.global_variables_initializer()
sess.run(init)

X = xTrain
Y = yTrain_onehot
batch_size = 3000

# Training cycle
for iteration in range(training_iteration):
    avg_cost = 0.
    total_batch = int(len(xTrain)/batch_size)

    # Loop over all batches
    for i in range(total_batch):
        batch_xs = xTrain[i*batch_size:(i*batch_size)+batch_size]
        batch_ys = yTrain[i*batch_size:(i*batch_size)+batch_size]                        
        batch_xs = batch_xs.reshape((batch_size, n_steps, n_input))        


        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            #print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
            #"{:.6f}".format(loss) + ", Training Accuracy= " + \
            #      "{:.5f}".format(acc))

print ("Optimization Finished!")

Optimization Finished!


<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Test hidden periods</div>

In [151]:
def getTestPredictions(X,Y):
    predictions=[]
    # Testing cycle
    total_batch = int(len(X)/batch_size)

    if np.mod(len(X),batch_size) != 0:
        # tf requires consistent inputs so need to pad
        rowsToPad=batch_size-np.mod(len(X),batch_size)
        padRows=np.zeros([rowsToPad,X.shape[1]])
        X = np.append(X,padRows,axis=0)

        padRows=np.zeros([rowsToPad,Y.shape[1]])
        Y = np.append(Y,padRows,axis=0)
        total_batch += 1

    # Loop over all batches
    for i in range(total_batch):
        
        batch_xs = X[i*batch_size:(i*batch_size)+batch_size]
        batch_ys = Y[i*batch_size:(i*batch_size)+batch_size]                
        
        test_len = np.shape(batch_xs)[0]
        batch_xs = batch_xs[:test_len].reshape((-1, n_steps, n_input))
        batch_ys = batch_ys[:test_len]

        p= 1*sess.run(pred, feed_dict={x: batch_xs, y: batch_ys})
        if predictions == []:
            predictions = p
        else:
            predictions= np.append(predictions,p,axis=0)
        
              
    n=len(predictions)-rowsToPad
    predictions=predictions[0:n]
    predictions = np.argmax(predictions,1)
    return predictions

<div style="background-color:#white; color:#008000; font-family: 'Courier New, Monospace;font-weight: bold">Test hidden periods</div>

In [152]:
predictions = getTestPredictions(xTest,yTest_onehot)
print(metrics.classification_report(yTest,predictions))  # Need to feed it yTest not yTest_OneHot here
print(metrics.confusion_matrix(yTest,predictions))
print("* Precision = labelled as x / how many were actually x in the ones that were labelled")
print("* Recall = labelled as x / how many were actually x in the dataset")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


             precision    recall  f1-score   support

          0       0.96      0.97      0.97     72531
          1       0.78      0.72      0.75      9360

avg / total       0.94      0.95      0.94     81891

[[70662  1869]
 [ 2591  6769]]
* Precision = labelled as x / how many were actually x in the ones that were labelled
* Recall = labelled as x / how many were actually x in the dataset


In [153]:
print(np.shape(yTest2_onehot),np.shape(predictions))
int(230182/batch_size)

(230182, 2) (81891,)


76

In [154]:
predictions = getTestPredictions(xTest2,yTest2_onehot)
print(metrics.classification_report(yTest2,predictions))  # Need to feed it yTest not yTest_OneHot here
print(metrics.confusion_matrix(yTest2,predictions))
print("* Precision = labelled as x / how many were actually x in the ones that were labelled")
print("* Recall = labelled as x / how many were actually x in the dataset")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


             precision    recall  f1-score   support

          0       0.97      0.98      0.98    212565
          1       0.73      0.66      0.69     17617

avg / total       0.95      0.95      0.95    230182

[[208208   4357]
 [  6015  11602]]
* Precision = labelled as x / how many were actually x in the ones that were labelled
* Recall = labelled as x / how many were actually x in the dataset
